In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Benchmarking STN vs What/Where model

- The eccentricites that the networks were trained on:

In [4]:
eccentricities = np.array([
    4.44444444, 8.88888889, 13.33333333, 17.77777778, 22.22222222, 26.66666667,
    31.11111111, 35.55555556, 40.
])

- The following What/Where accuracy statistics were recuperated from the original What/Where repo, from this [notebook](https://github.com/laurentperrinet/WhereIsMyMNIST/blob/master/notebooks/4_results_figure_contrast.ipynb) specifically

In [3]:
what_where_stats = {
    0.3: {
        0:
        np.array(
            [0.093, 0.115, 0.103, 0.103, 0.131, 0.182, 0.342, 0.455, 0.528]),
        1:
        np.array(
            [0.164, 0.234, 0.252, 0.318, 0.368, 0.397, 0.466, 0.512, 0.538]),
        2:
        np.array(
            [0.197, 0.258, 0.341, 0.325, 0.411, 0.436, 0.47, 0.501, 0.538]),
        3:
        np.array(
            [0.21, 0.261, 0.323, 0.357, 0.416, 0.438, 0.453, 0.509, 0.506]),
        4:
        np.array(
            [0.224, 0.282, 0.326, 0.33, 0.359, 0.444, 0.453, 0.498, 0.484]),
        5:
        np.array(
            [0.204, 0.288, 0.319, 0.341, 0.347, 0.404, 0.462, 0.47, 0.476])
    },
    0.5: {
        0:
        np.array(
            [0.111, 0.105, 0.127, 0.108, 0.151, 0.306, 0.478, 0.699, 0.823]),
        1:
        np.array(
            [0.422, 0.537, 0.566, 0.625, 0.66, 0.717, 0.762, 0.811, 0.833]),
        2:
        np.array(
            [0.493, 0.608, 0.649, 0.678, 0.728, 0.776, 0.78, 0.799, 0.804]),
        3:
        np.array(
            [0.502, 0.607, 0.657, 0.706, 0.715, 0.765, 0.79, 0.789, 0.792]),
        4:
        np.array(
            [0.526, 0.631, 0.649, 0.721, 0.719, 0.747, 0.791, 0.792, 0.789]),
        5:
        np.array(
            [0.523, 0.602, 0.651, 0.686, 0.737, 0.752, 0.787, 0.78, 0.805])
    },
    0.7: {
        0:
        np.array(
            [0.111, 0.112, 0.102, 0.104, 0.159, 0.287, 0.532, 0.756, 0.899]),
        1:
        np.array([0.628, 0.672, 0.714, 0.724, 0.777, 0.782, 0.859, 0.879,
                  0.9]),
        2:
        np.array(
            [0.703, 0.765, 0.786, 0.824, 0.836, 0.86, 0.879, 0.879, 0.872]),
        3:
        np.array([0.729, 0.781, 0.809, 0.83, 0.838, 0.857, 0.854, 0.89,
                  0.875]),
        4:
        np.array(
            [0.714, 0.777, 0.814, 0.826, 0.856, 0.869, 0.876, 0.884, 0.877]),
        5:
        np.array([0.74, 0.8, 0.823, 0.833, 0.842, 0.854, 0.872, 0.866, 0.886])
    }
}

- The performance of the pure STN_128x128 was recuperated from the notebook **"STN_128x128"** where the network was trained and tested.

In [5]:
stn = {
    0.7:
    np.array([
        0.9794, 0.9788, 0.9784, 0.9798, 0.9788, 0.9777, 0.9776, 0.9738, 0.9727
    ])
}